In [72]:
set_option(rational_to_decimal=True)

# imports

## pip

In [73]:
!pip install anchor-exp
!pip install pmlb
!pip install z3-solver

## import

In [74]:
import numpy as np
import pandas as pd

from anchor import utils
from anchor import anchor_tabular
import sklearn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from pmlb import fetch_data
from z3 import *

# model

In [75]:
def feature_constraints_expression(X):
    constraints = []

    for i in range(X.shape[1]):
        feature_values = X[:, i]
        min_val, max_val = feature_values.min(), feature_values.max()

        x = Real(f'x{i}')
        min = RealVal(min_val)
        max = RealVal(max_val)

        constraint = And(min <= x, x <= max)
        constraints.append(constraint)

    return And(*constraints)

In [76]:
def tree_paths_expression(tree, tree_index, class_index):
    tree_ = tree.tree_
    feature = tree_.feature
    threshold = tree_.threshold
    value = tree_.value

    paths = []
    o = Real(f'o_{tree_index}_{class_index}')

    def traverse(node, path_conditions):

        if feature[node] == -2:
            leaf_value = value[node][0][0]
            path_formula = And(path_conditions)
            implication = Implies(path_formula, o == leaf_value)
            paths.append(implication)
        else:

            x = Real(f'x{feature[node]}')
            left_condition = x <= threshold[node]
            right_condition = x > threshold[node]
            traverse(tree_.children_left[node],
                     path_conditions + [left_condition])
            traverse(tree_.children_right[node],
                     path_conditions + [right_condition])

    traverse(0, [])
    return And(*paths)

In [77]:
def model_trees_expression(model):
    formulas = []
    for i, estimators in enumerate(model.estimators_):
        for class_index, estimator in enumerate(estimators):
            formula = tree_paths_expression(estimator, i, class_index)
            formulas.append(formula)
    return And(*formulas)

In [78]:
def decision_function_expression(model, x):
    learning_rate = model.learning_rate
    estimators = model.estimators_
    n_classes = 1 if model.n_classes_ <= 2 else model.n_classes_

    decision = model.decision_function(x)
    predicted_class = model.predict(x)[0]

    estimator_results = []
    for estimator in estimators:
        class_predictions = [tree.predict(x) for tree in estimator]
        estimator_results.append(class_predictions)

    estimator_sum = np.sum(estimator_results, axis=0) * learning_rate
    init_value = decision - estimator_sum.T

    equation_list = []
    for class_number in range(n_classes):
        estimator_list = []
        for estimator_number in range(len(estimators)):
            o = Real(f"o_{estimator_number}_{class_number}")
            estimator_list.append(o)
        equation_o = Sum(estimator_list) * learning_rate + init_value[0][class_number]
        equation_list.append(equation_o)

    if n_classes <= 2:
        if predicted_class == 0:
            final_equation = equation_list[0] < 0
        else:
            final_equation = equation_list[0] > 0
    else:
        compare_equation = []
        for class_number in range(n_classes):
            if predicted_class != class_number:
                compare_equation.append(
                    equation_list[predicted_class] > equation_list[class_number]
                )
        final_equation = compare_equation

    return And(final_equation)

In [79]:
def instance_expression(instance):
    formula = [Real(f'x{i}') == value for i, value in enumerate(instance)]
    return formula

In [80]:
def is_proved(f):
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat:
        return True
    else:
        # print(s.model())
        return False

In [81]:
def explain(I, T, D, model, reorder):
    X = I.copy()
    T_s = simplify(T)
    D_s = simplify(D)

    importances = model.feature_importances_
    if reorder == 'asc':
        sorted_feature_indices = np.argsort(importances)
        X = [X[i] for i in sorted_feature_indices]
    elif reorder == 'desc':
        sorted_feature_indices = np.argsort(np.flip(importances))
        X = [X[i] for i in sorted_feature_indices]

    for feature in X.copy():
        X.remove(feature)

        # prove(Implies(And(And(X), T), D))
        if is_proved(Implies(And(And(X), T_s), D_s)):
            continue
            # print('proved')
        else:
            # print('not proved')
            X.append(feature)

    return X

In [82]:
class Explainer:
    def __init__(self, model, data):
        self.model = model
        self.data = data
        self.T_constraints = feature_constraints_expression(self.data)
        self.T_model = model_trees_expression(self.model)
        self.T = And(self.T_model, self.T_constraints)

    def explain(self, instance, reorder='asc'):
        self.D = decision_function_expression(self.model, [instance])
        self.I = instance_expression(instance)

        return explain(self.I, self.T, self.D, self.model, reorder)

In [143]:
class ExplainerCompleter():
  def __init__(self, model, anchor_explainer, data, round):
    self.model = model

    # anchor
    # explain instance > matriz > expressions
    self.anchor_explainer = anchor_explainer

    # model
    # T
    self.T_constraints = feature_constraints_expression(data)
    self.T_model = model_trees_expression(self.model)
    self.T = And(self.T_model, self.T_constraints)

  def explain_instance(self, instance, verbose=False):
    opt = Optimize()

    # anchor matrix > expressions
    exp = anchor_explainer.explain_instance(instance, gb_iris.predict, threshold=0.95)
    anchor_matrix = []
    for name in exp.names():
      tokens = name.split(' ')
      for operator in ['<=', '>=', '==', '<', '>']:
        if operator in name:
          parts = name.split(operator)
          if len(parts) == 2:
            anchor_matrix.append([parts[0].strip(), operator, parts[1].strip()])
            break
    # unir com o código de cima para simplificar
    anchor_expressions = []
    for row in anchor_matrix:
      feature = Real(row[0])
      if row[1] == '<=':
        expression = feature <= float(row[2])
      elif row[1] == '>=':
        expression = feature >= float(row[2])
      elif row[1] == '<':
        expression = feature < float(row[2])
      elif row[1] == '>':
        expression = feature > float(row[2])
      anchor_expressions.append(expression)
    # print(anchor_expressions, len(anchor_expressions) == len(anchor_matrix))
    self.anchor_expressions = anchor_expressions
    opt.add(anchor_expressions)

    # delta
    # delta >= 0
    # todas as features que não estao no anchor > fazer as igualdades delta
    anchor_variables = []
    for formula in anchor_expressions:
      anchor_variables.append(str(formula.arg(0)))

    feature_names = [f'x{i}' for i in range(instance.shape[0])]
    opt.add(delta >= 0)
    for i, var in enumerate(feature_names):
      if var not in anchor_variables: # and importance_dic[var] != 0:
        z3_var = Real(var)
        opt.add((instance[i]) - delta <= z3_var, z3_var <= (instance[i]) + delta)
        # print(f'{instance[i]} - {delta} <= {var}, {var} <= {instance[i]} + {delta}')

    # not D
    self.D = decision_function_expression(self.model, [instance])

    # model
    opt.add(self.T)
    opt.add(Not(self.D))

    # minimize delta
    opt.minimize(delta)
    if opt.check() == sat:
      if verbose:
        for var in opt.model():
          print(var, '=', opt.model()[var])
      if opt.model().eval(delta) == 0:
        print('delta = 0')
      else:
        print(f"\ndelta: {opt.model().eval(delta)}")
    else:
      print("problema inviavel / explicação correta")

In [123]:
def complete_anchor_explainer(anchor_exp_expressions, delta_round, explainer, instance, feature_names, x_values_dic):
  opt = Optimize()

  delta = Real('delta')

  # anchor
  opt.add(anchor_exp_expressions)
  anchor_variables = []
  for formula in anchor_exp_expressions:
    anchor_variables.append(str(formula.arg(0)))

  # delta
  opt.add(delta >= 0)
  for var in feature_names:
    if var not in anchor_variables: # and importance_dic[var] != 0:
      z3_var = Real(var)
      opt.add((x_values_dic[var]) - delta <= z3_var, z3_var <= (x_values_dic[var]) + delta)
      # print(f'{x_values_dic[var]} - {delta} <= {var}, {var} <= {x_values_dic[var]} + {delta}')

  # model
  explainer.explain(instance)
  opt.add(explainer.T_constraints)
  opt.add(explainer.T_model)
  opt.add(Not(explainer.D))

  opt.minimize(delta)
  if opt.check() == sat:
    for var in opt.model():
      print(var, '=', opt.model()[var])
    if opt.model().eval(delta) == 0:
      print('delta == 0')
    else:
      print(f"\ndelta: {opt.model().eval(delta)}")
  else:
    print("problema inviavel / explicação correta")

  return

In [85]:
def complete_anchor_explainer_d(anchor_exp_expressions, delta_round, explainer, instance, feature_names, x_values_dic):
  opt = Optimize()


  # anchor
  opt.add(anchor_exp_expressions)
  anchor_variables = []
  for formula in anchor_exp_expressions:
    anchor_variables.append(str(formula.arg(0)))

  # delta
  for i, var in enumerate(feature_names):
    if var not in anchor_variables: # and importance_dic[var] != 0:
      z3_var = Real(var)
      delta = Real(f'delta{i}')
      opt.add(delta >= 0)
      opt.add((x_values_dic[var]) - delta <= z3_var, z3_var <= (x_values_dic[var]) + delta)
      # print(f'{x_values_dic[var]} - {delta} <= {var}, {var} <= {x_values_dic[var]} + {delta}')

  # model
  explainer.explain(instance)
  opt.add(explainer.T_constraints)
  opt.add(explainer.T_model)
  opt.add(Not(explainer.D))

  for i in range(len(feature_names)):
    delta = Real(f'delta{i}')
    opt.minimize(delta)
  # opt.minimize(delta)
  if opt.check() == sat:
    for var in opt.model():
      print(var, '=', opt.model()[var])
    if opt.model().eval(delta) == 0:
      print('anchor correct')
    else:
      print(f"\ndelta: {opt.model().eval(delta)}")
  else:
    print("Problema inviável!")

  return

# tests

In [86]:
gb_iris = GradientBoostingClassifier(n_estimators=1, max_depth=1, random_state = 101)

iris = load_iris()
X_iris, y_iris = iris.data, iris.target

# deixa binario
filter_indices = np.where(np.isin(y_iris, [0, 1]))[0]
X_iris = X_iris[filter_indices]
y_iris = y_iris[filter_indices]

X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(
    X_iris, y_iris, test_size=0.2, random_state=101)

gb_iris.fit(X_iris_train, y_iris_train)

print('Train', sklearn.metrics.accuracy_score(y_iris_train, gb_iris.predict(X_iris_train)))
print('Test', sklearn.metrics.accuracy_score(y_iris_test, gb_iris.predict(X_iris_test)))

gb_iris.predict(X_iris)

Train 1.0
Test 1.0


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [87]:
range(len(iris.feature_names))

range(0, 4)

In [105]:
iris_features_x = [f'x{i}' for i in range(len(iris.feature_names))]
iris_features_x

['x0', 'x1', 'x2', 'x3']

In [111]:
for i, var in enumerate(iris_features_x):
  print(var, X_iris[idx][i])

x0 5.1
x1 3.5
x2 1.4
x3 0.2


In [89]:
idx = 0
iris_features_x = [f'x{i}' for i in range(len(iris.feature_names))]

anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    gb_iris.classes_,
    iris_features_x,
    X_iris_train,
    categorical_names={})

print('Prediction: ', anchor_explainer.class_names[gb_iris.predict(X_iris[idx].reshape(1, -1))[0]])
exp = anchor_explainer.explain_instance(X_iris[idx], gb_iris.predict, threshold=0.95)

anchor_matrix = []

for name in exp.names():
  tokens = name.split(' ')

  for operator in ['<=', '>=', '==', '<', '>']:
    if operator in name:
      parts = name.split(operator)
      if len(parts) == 2:
        anchor_matrix.append([parts[0].strip(), operator, parts[1].strip()])
        break
anchor_matrix

Prediction:  0


[['x3', '<=', '0.20']]

In [90]:
anchor_expressions = []
for row in anchor_matrix:
  feature = Real(row[0])
  if row[1] == '<=':
    expression = feature <= float(row[2])
  elif row[1] == '>=':
    expression = feature >= float(row[2])
  anchor_expressions.append(expression)
print(anchor_expressions, len(anchor_expressions) == len(anchor_matrix))

[x3 <= 0.2] True


In [91]:
explainer = Explainer(gb_iris, X_iris_train)

In [92]:
instance_explanation = explainer.explain(X_iris_train[idx], 'asc')
instance_explanation

[x3 == 1.5]

In [93]:
print(anchor_expressions)

[x3 <= 0.2]


In [94]:
importances = gb_iris.feature_importances_
importance_dic = {feature: importance for feature, importance in zip(iris_features_x, importances)}
importance_dic

{'x0': 0.0, 'x1': 0.0, 'x2': 0.0, 'x3': 1.0}

In [95]:
x_values_dic = {feature: value for feature, value in zip(iris_features_x, X_iris_train[idx])}
x_values_dic

{'x0': 5.9, 'x1': 3.0, 'x2': 4.2, 'x3': 1.5}

In [96]:
opt = Optimize()
delta = Real('delta')

# Adiciona a restrição delta > 0
opt.add(delta >= 0)

anchor_variables = []
for formula in anchor_expressions:
  anchor_variables.append(str(formula.arg(0)))

In [97]:
instance = X_iris_train[idx]

for var in iris_features_x:
  if var not in anchor_variables and importance_dic[var] != 0:
    print(f'{x_values_dic[var]} - {delta} <= {var}, {var} <= {x_values_dic[var]} + {delta}')

In [98]:
instance = X_iris_train[idx]
formula = [Real(f'x{i}') == value for i, value in enumerate(instance)]
print(formula)

[x0 == 5.9, x1 == 3, x2 == 4.2, x3 == 1.5]


In [121]:
complete_anchor_explainer(anchor_expressions, 0, explainer, X_iris_train[idx], iris_features_x, x_values_dic)

x2 = 4.2
x1 = 3
o_0_0 = -2.1621621621?
delta = 0
x3 = 0.1
x0 = 5.9
anchor correct


In [122]:
complete_anchor_explainer_d(anchor_expressions, 0, explainer, X_iris_train[idx], iris_features_x, x_values_dic)

o_0_0 = -2.1621621621?
delta1 = 0
x2 = 4.2
x1 = 3
delta0 = 0
x0 = 5.9
x3 = 0.1
delta2 = 0

delta: delta3


In [101]:
print(explainer.T_model)

And(And(Implies(And(x3 <= 0.75), o_0_0 == -2.1621621621?),
        Implies(And(x3 > 0.75), o_0_0 == 1.8604651162?)))


In [144]:
expcomp = ExplainerCompleter(gb_iris, anchor_explainer, X_iris, 0)

In [152]:
expcomp.explain_instance(X_iris_train[3], verbose=True)

x2 = 3.3
x1 = 2
o_0_0 = -2.1621621621?
delta = 0
x3 = 0.75
x0 = 4.9
delta = 0


In [153]:
expcomp.anchor_expressions

[x3 > 0.2, x1 <= 2.8]

In [155]:
print(expcomp.D)

And((o_0_0)*0.1 + 0.1502822030? > 0)


In [154]:
print(expcomp.T)

And(And(And(Implies(And(x3 <= 0.75),
                    o_0_0 == -2.1621621621?),
            Implies(And(x3 > 0.75), o_0_0 == 1.8604651162?))),
    And(And(4.3 <= x0, 7 >= x0),
        And(2 <= x1, 4.4 >= x1),
        And(1 <= x2, 5.1 >= x2),
        And(0.1 <= x3, 1.8 >= x3)))


In [136]:
expcomp = ExplainerCompleter(gb_iris, anchor_explainer, X_iris, 0)

for i in range(len(X_iris_train)):
  expcomp.explain_instance(X_iris_train[i])

problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema inviavel / explicação correta
problema inviavel / explicação correta
problema inviavel / explicação correta
delta = 0
problema invia